In [1]:
from __future__ import print_function
import keras, pickle
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten
from keras import backend as K

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
infile = open('../data/data8.pickle','rb')
new_dict = pickle.load(infile)
y_train = new_dict['y_train']
y_test = new_dict['y_test']
x_train = new_dict['x_train'][:,0,:,:]
x_test = new_dict['x_test'][:,0,:,:]
y_train

array([37, 27, 38, ..., 11, 14, 30])

In [3]:
batch_size = 100
epochs = 4
img_rows = x_train[0].shape[0]
img_cols = x_train[0].shape[1]
num_chanels = 1
num_classes = len(set(y_train))

In [4]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
print(f"We have data prepared for backend [{keras.backend.backend()}]")
print(input_shape)

We have data prepared for backend [tensorflow]
(32, 32, 1)


In [5]:
from keras.utils import to_categorical

# Convert class vectors to binary class matrices, like get_dummies on pandas
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [6]:
## This is the neural network proposed architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='relu', input_shape=(input_shape)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
loss_function = keras.losses.categorical_crossentropy
optimizer = keras.optimizers.Adadelta()
model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

In [8]:
history = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Instructions for updating:
Use tf.cast instead.
Train on 86989 samples, validate on 12630 samples
Epoch 1/4
86989/86989 [==============================] - 229s 3ms/step - loss: 0.5962 - accuracy: 0.8367 - val_loss: 0.1779 - val_accuracy: 0.9523
Epoch 2/4
86989/86989 [==============================] - 229s 3ms/step - loss: 0.1221 - accuracy: 0.9651 - val_loss: 0.1555 - val_accuracy: 0.9592
Epoch 3/4
86989/86989 [==============================] - 179s 2ms/step - loss: 0.0724 - accuracy: 0.9790 - val_loss: 0.1427 - val_accuracy: 0.9654
Epoch 4/4
86989/86989 [==============================] - 142s 2ms/step - loss: 0.0517 - accuracy: 0.9848 - val_loss: 0.1304 - val_accuracy: 0.9690


In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.13037994603297554
Test accuracy: 0.9690419435501099


In [10]:
%matplotlib inline
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

KeyError: 'acc'

In [ ]:
# Plot confusion matrix to have a more detailed view of cross-class missclassifications
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix
from matplotlib.pyplot import figure

## Use argmax to project output probabilites as class index label
proba = model.predict(x_test)
print(proba[0])
y_pred = np.argmax(proba, axis=1)
print(y_pred[0])
y_t = np.argmax(y_test, axis=1)

cm = confusion_matrix(y_t, y_pred)

figure(figsize=(15,15))
sns.heatmap(cm, annot=True)
plt.xlabel("GT")
plt.ylabel("Predicted")
plt.title(f"Test data = {len(y_pred)} samples")
plt.show()

In [ ]:
#model.save("data/model_train.h5")
model.save_weights('../data/model/model_trained_weights.h5')
